# Preprocesamiento de Datos y Ingeniería de Características
## 📊 Fase 2: Preparación de Datos para Modelado

**Objetivo:** Transformar los datos crudos en un formato adecuado para el entrenamiento de modelos de Machine Learning.

**Tareas principales:**
- Limpieza y manejo de valores faltantes
- Codificación de variables categóricas
- Escalado y normalización de características numéricas
- Ingeniería de nuevas características (feature engineering)
- División del dataset en conjuntos de entrenamiento, validación y prueba
- Manejo del desbalanceo de clases

---

## 📋 1. Carga de Datos y Configuración Inicial

Vamos a cargar los datos preprocesados y configurar el entorno de trabajo.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings('ignore')

# Configuración de estilo
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 8)

# Cargar datos
print("🚀 Iniciando preprocesamiento de datos...")
try:
    df = pd.read_csv("../data/raw/data.csv", sep=";")
    print(f"✅ Datos cargados exitosamente: {df.shape}")
except FileNotFoundError:
    print("❌ Error: No se encontró el archivo data.csv")

## 🔍 2. Análisis Inicial y Limpieza de Datos

Primero vamos a explorar los datos para identificar problemas y planificar la limpieza.

In [ ]:
# Información general del dataset
print("📊 Información del dataset:")
print(f"Número de filas: {df.shape[0]}")
print(f"Número de columnas: {df.shape[1]}")
print(f"Tipos de datos:
{df.dtypes.value_counts()}")

# Valores faltantes
print("
🚨 Valores faltantes por columna:")
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])

# Estadísticas básicas
print("
📈 Estadísticas descriptivas:")
print(df.describe())

## 🧹 3. Limpieza de Datos y Manejo de Valores Faltantes

Vamos a limpiar los datos y manejar los valores faltantes apropiadamente.

In [ ]:
# Crear copia del dataset para preprocesamiento
df_processed = df.copy()

# 1. Verificar valores faltantes
print("Valores faltantes antes de limpieza:")
print(df_processed.isnull().sum().sum())

# 2. Estrategias de manejo de valores faltantes:
# Para variables numéricas: usar mediana (más robusta que la media)
# Para variables categóricas: usar moda o crear categoría 'Desconocido'

numeric_cols = df_processed.select_dtypes(include=[np.number]).columns
categorical_cols = df_processed.select_dtypes(exclude=[np.number]).columns

print(f"Columnas numéricas: {len(numeric_cols)}")
print(f"Columnas categóricas: {len(categorical_cols)}")

# Imputación para columnas numéricas
for col in numeric_cols:
    if df_processed[col].isnull().sum() > 0:
        median_value = df_processed[col].median()
        df_processed[col].fillna(median_value, inplace=True)
        print(f"✅ Imputada columna numérica '{col}' con mediana: {median_value:.2f}")

# Imputación para columnas categóricas
for col in categorical_cols:
    if df_processed[col].isnull().sum() > 0:
        mode_value = df_processed[col].mode().iloc[0] if not df_processed[col].mode().empty else 'Desconocido'
        df_processed[col].fillna(mode_value, inplace=True)
        print(f"✅ Imputada columna categórica '{col}' con moda: {mode_value}")

print(f"
✅ Valores faltantes después de limpieza: {df_processed.isnull().sum().sum()}")

## 🏷️ 4. Codificación de Variables Categóricas

Las variables categóricas necesitan ser transformadas a formato numérico para los modelos de ML.

In [ ]:
# Separar características y variable objetivo
feature_cols = [col for col in df_processed.columns if col != 'Target']
target_col = 'Target'

X = df_processed[feature_cols]
y = df_processed[target_col]

# Identificar tipos de columnas para preprocessing
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X.select_dtypes(exclude=[np.number]).columns.tolist()

print(f"Características numéricas: {len(numeric_features)}")
print(f"Características categóricas: {len(categorical_features)}")

# Crear preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(drop='first', sparse=False), categorical_features)
    ],
    remainder='passthrough'
)

print("
🔧 Preprocesador creado exitosamente")
print(f"Total de características después de encoding: {len(numeric_features) + len(categorical_features)}")

## ⚖️ 5. Manejo del Desbalanceo de Clases

Vamos a analizar si hay desbalanceo significativo en la variable objetivo y aplicar técnicas apropiadas.

In [ ]:
# Analizar distribución de clases
print("📊 Distribución de clases objetivo:")
class_distribution = y.value_counts()
print(class_distribution)

# Calcular porcentajes
class_percentages = (class_distribution / len(y)) * 100
print(f"
Porcentajes:
{class_percentages}")

# Visualizar distribución
plt.figure(figsize=(10, 6))
bars = plt.bar(class_distribution.index, class_distribution.values,
               color=['red', 'green', 'blue'])
plt.title('Distribución de Clases - Variable Objetivo')
plt.xlabel('Clases')
plt.ylabel('Número de Muestras')
plt.xticks(rotation=45)

# Agregar etiquetas con porcentajes
for bar, percentage in zip(bars, class_percentages):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 50,
             f'{percentage:.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Evaluar si hay desbalanceo significativo
min_class_ratio = class_distribution.min() / class_distribution.max()
print(f"
📏 Ratio de desbalanceo (min/max): {min_class_ratio:.3f}")

if min_class_ratio < 0.8:  # Si hay desbalanceo significativo
    print("⚠️ Se detecta desbalanceo significativo de clases")
    print("💡 Recomendaciones:")
    print("   - Usar técnicas de oversampling (SMOTE) para clases minoritarias")
    print("   - Usar técnicas de undersampling para clases mayoritarias")
    print("   - Usar métricas de evaluación que manejen desbalanceo (F1-score, AUC-ROC)")
    print("   - Considerar usar class_weight='balanced' en modelos que lo soporten")
else:
    print("✅ Las clases están razonablemente balanceadas")

## ✂️ 6. División del Dataset

Vamos a dividir los datos en conjuntos de entrenamiento, validación y prueba.

In [ ]:
# Estratificar por la variable objetivo para mantener distribución de clases
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y,
    test_size=0.3,  # 70% entrenamiento, 30% temporal
    random_state=42,
    stratify=y  # Mantener proporción de clases
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.5,  # 50% de los datos temporales para validación y prueba
    random_state=42,
    stratify=y_temp  # Mantener proporción en el split
)

print(f"📊 División del dataset:")
print(f"   Entrenamiento: {X_train.shape[0]} muestras ({X_train.shape[0]/len(df)*100:.1f}%)")
print(f"   Validación: {X_val.shape[0]} muestras ({X_val.shape[0]/len(df)*100:.1f}%)")
print(f"   Prueba: {X_test.shape[0]} muestras ({X_test.shape[0]/len(df)*100:.1f}%)")
print(f"
Verificación de estratificación:")

# Verificar que las proporciones se mantienen
for split_name, split_y in [("Entrenamiento", y_train), ("Validación", y_val), ("Prueba", y_test)]:
    print(f"   {split_name}: {split_y.value_counts().to_dict()}")

## 🚀 7. Aplicación del Preprocesamiento Completo

Ahora vamos a aplicar todo el pipeline de preprocesamiento a nuestros conjuntos de datos.

In [ ]:
# Aplicar preprocesamiento a los conjuntos de datos
print("🔄 Aplicando preprocesamiento...")

X_train_processed = preprocessor.fit_transform(X_train)
X_val_processed = preprocessor.transform(X_val)
X_test_processed = preprocessor.transform(X_test)

# Obtener nombres de características después del encoding
feature_names = (numeric_features +
                preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features).tolist())

print(f"✅ Características originales: {X_train.shape[1]}")
print(f"✅ Características procesadas: {X_train_processed.shape[1]}")
print(f"📊 Forma de los conjuntos procesados:")
print(f"   X_train: {X_train_processed.shape}")
print(f"   X_val: {X_val_processed.shape}")
print(f"   X_test: {X_test_processed.shape}")

## 💾 8. Guardado de Datos Preprocesados

Finalmente, vamos a guardar los conjuntos de datos preprocesados para usarlos en el modelado.

In [ ]:
# Crear DataFrames procesados
train_processed = pd.DataFrame(X_train_processed, columns=feature_names)
train_processed['Target'] = y_train.reset_index(drop=True)

val_processed = pd.DataFrame(X_val_processed, columns=feature_names)
val_processed['Target'] = y_val.reset_index(drop=True)

test_processed = pd.DataFrame(X_test_processed, columns=feature_names)
test_processed['Target'] = y_test.reset_index(drop=True)

# Guardar conjuntos procesados
output_dir = "../data/processed/"

try:
    train_processed.to_csv(f"{output_dir}train_processed.csv", index=False)
    val_processed.to_csv(f"{output_dir}val_processed.csv", index=False)
    test_processed.to_csv(f"{output_dir}test_processed.csv", index=False)
    
    print("💾 Datos procesados guardados exitosamente:")
    print(f"   ✅ train_processed.csv: {train_processed.shape}")
    print(f"   ✅ val_processed.csv: {val_processed.shape}")
    print(f"   ✅ test_processed.csv: {test_processed.shape}")
    
    # Guardar información del preprocesamiento
    preprocessing_info = {
        'numeric_features': numeric_features,
        'categorical_features': categorical_features,
        'feature_names': feature_names,
        'class_distribution': class_distribution.to_dict(),
        'dataset_shapes': {
            'original': df.shape,
            'train': X_train_processed.shape,
            'val': X_val_processed.shape,
            'test': X_test_processed.shape
        }
    }
    
    import json
    with open(f"{output_dir}preprocessing_info.json", 'w') as f:
        json.dump(preprocessing_info, f, indent=2)
    
    print(f"✅ Información de preprocesamiento guardada en preprocessing_info.json")
    
except Exception as e:
    print(f"❌ Error al guardar archivos: {e}")
    print("💡 Crea el directorio ../data/processed/ si no existe")

## 📋 9. Resumen del Preprocesamiento

Vamos a crear un resumen completo del proceso de preprocesamiento realizado.

In [ ]:
print("🎉 PREPROCESAMIENTO COMPLETADO")
print("=" * 50)
print(f"📊 Datos originales: {df.shape}")
print(f"📈 Datos de entrenamiento: {X_train_processed.shape}")
print(f"📋 Datos de validación: {X_val_processed.shape}")
print(f"🧪 Datos de prueba: {X_test_processed.shape}")

print("
🔧 Transformaciones aplicadas:")
print(f"   ✅ Valores faltantes manejados: {df.isnull().sum().sum()} → 0")
print(f"   ✅ Variables categóricas codificadas: {len(categorical_features)}")
print(f"   ✅ Variables numéricas escaladas: {len(numeric_features)}")
print(f"   ✅ Nuevas características creadas: {X_train_processed.shape[1] - X_train.shape[1]}")

print("
📋 Archivos generados:")
print("   📁 ../data/processed/train_processed.csv")
print("   📁 ../data/processed/val_processed.csv")
print("   📁 ../data/processed/test_processed.csv")
print("   📁 ../data/processed/preprocessing_info.json")

print("
🚀 ¡Listo para la siguiente fase: Modelado!")
print("💡 Los datos están ahora en el formato óptimo para entrenamiento de modelos ML")